In [ ]:
import numpy as np
import math

In [ ]:
# scara
def scara_invkin(x,y,z,d1,d2):
    # using formulae from the textbook
    r = abs((x**2+y**2-d1**2-d2**2)/(2*d1*d2))
    theta2 = np.arctan(np.sqrt(abs(1-r**2))/r)
    theta1 = np.arctan(y/x) - np.arctan((d2*np.sin(theta2))/(d1+d2*np.cos(theta2)))
    d3 = -z
    # converting from radians to degrees
    theta1_deg = theta1*180/np.pi
    theta2_deg = theta2*180/np.pi
    print("\nInverse kinematics solution:")
    print(f"theta1 = {theta1_deg} deg")
    print(f"theta2 = {theta2_deg} deg")
    return theta1, theta2

# standford
def stanford_inverse_kinematics(endeffector_position,lengthsoflinks):
    theta1 = np.arctan(endeffector_position[1]/endeffector_position[0])
    r = np.sqrt(endeffector_position[0]**2 + endeffector_position[1]**2)
    s = endeffector_position[2] - lengthsoflinks[0]
    theta2 = np.arctan(s/r)
    d3 = np.sqrt(r**2 + s**2) - lengthsoflinks[1]
    print("First Solution: \n", "Theta1 = ", theta1, "\n Theta2 =", theta2,"\n Extension: ", d3, "\n")
    print("Second Solution: \n", "Theta1 = ", np.pi + theta1, "\n Theta2 =", np.pi - theta2,"\n Extension:", d3)
    return theta1, theta2,d3

# inverse kinematics for puma
def puma_inverse_kinematics(l1,l2,l3,xc,yc,zc):
    theta1 = math.atan2(yc,xc)
    D = (xc*xc+yc*yc+(zc-l1)*(zc-l1)-l2*l2-l3*l3)/(2*l2*l3)
    if D>=1 or D<=-1:
        print("singular configuration")
    if D>1 or D<-1:
        print("outside workspace")   
    theta3 = (math.atan2((-math.sqrt(1-D*D)),D))
    theta2 = math.atan2(zc-l1,(math.sqrt(xc*xc+yc*yc)))-math.atan2((l3*math.sin(theta3)),(l2+l3*math.cos(theta3)))
    print("Theta1 = ",np.rad2deg(theta1),"\nTheta2 = ", np.rad2deg(theta2), "\nTheta3 = ", np.rad2deg(theta3))

    return theta1,theta2,theta3

def atransformation(LinkParameters):
  a = LinkParameters[0]
  d = LinkParameters[2]
  alpha = LinkParameters[1]
  theta = LinkParameters[3]
  A = np.array([[[np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
                  [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
                  [0, np.sin(alpha), np.cos(alpha), d],
                  [0, 0, 0, 1]]])
  return (A)

# Homogenous Matrices Forward Kinematics from DH
def forward_kinematics_homogenous_matrix(DH):
    A = np.identity(4)
    n = np.shape(DH)[0]

    for i in range(n):
        Anext = atransformation(DH[i])
        A = np.matmul(A,Anext)

    return A

class Robot():
  def __init__(self,robot_type):
    self.robot_type = robot_type
  
  def calc_invk_and_fk(self):
    if self.robot_type == 'SCARA':
      print("Enter the length of links d1 and d2:")
      d1 = float(input("d1 = "))
      d2 = float(input("d2 = "))
      n = int(input("Enter the no. of end effector points:"))
      for i in range(n):
        print(f"Enter x, y, z cordinate of point {i+1}:")
        x = float(input("x = "))
        y = float(input("y = "))
        z = float(input("z = "))
        theta1, theta2 = scara_invkin(x,y,z,d1,d2)
        print("\nForward kinematics solution:")
        link_parameters = [[d1, np.pi/2, 0, theta1],[d2, -np.pi/2, 0, theta2]]
        T = forward_kinematics_homogenous_matrix(link_parameters)
        print("Final transformation matrix: ")
        print(T)
    elif self.robot_type == "STANFORD":
      print("Enter the length of links d1 and d2:")
      d1 = float(input("d1 = "))
      d2 = float(input("d2 = "))
      n = int(input("Enter the no. of end effector points:"))
      for i in range(n):
        print(f"Enter x, y, z cordinate of point {i+1}:")
        x = float(input("x = "))
        y = float(input("y = "))
        z = float(input("z = "))
        print("\nInverse kinematics solution: ")
        theta1, theta2, d3 = stanford_inverse_kinematics((x,y,z),(d1,d2))
        print("\nForward kinematics solution:")
        LinkParameters = [[0, -np.pi/2, 0, theta1],[0, -np.pi/2, 0, theta2]]
        T = forward_kinematics_homogenous_matrix(LinkParameters)
        print("Final transformation matrix: ")
        print(T)
    elif self.robot_type == "PUMA":
      print("Enter the length of links l1, l2 and l3:")
      l1 = float(input("l1 = "))
      l2 = float(input("l2 = "))
      l3 = float(input("l3 = "))
      n = int(input("Enter the no. of end effector points:"))
      for i in range(n):
        print(f"Enter x, y, z cordinate of point {i+1}:")
        x = float(input("x = "))
        y = float(input("y = "))
        z = float(input("z = "))
        print("\nInverse kinematics solution: ")
        theta1, theta2, theta3 = puma_inverse_kinematics(l1, l2, l3, x, y, z)
        print("\nforward kinematics solution")
        #LinkParameters = [a, alpha, d, theta]
        LinkParameters = [[l1, 0, 0, theta1], [l2, -np.pi/2, 0, theta2], [0, 0, 0, theta3]]
        T = forward_kinematics_homogenous_matrix(LinkParameters)
        print("Final transformation matrix: ")
        print(T)

In [ ]:
scara = Robot("SCARA")
scara.calc_invk_and_fk()

Enter the length of links d1 and d2:
d1 = 0.25
d2 = 0.25
Enter the no. of end effector points:1
Enter x, y, z cordinate of point 1:
x = 0.45
y = 0.075
z = 0.1

Inverse kinematics solution:
theta1 = -14.696515095304628 deg
theta2 = 48.31767460666049 deg

Forward kinematics solution:
Final transformation matrix: 
[[[ 6.43243318e-01  2.53699112e-01 -7.22409021e-01  4.02631626e-01]
  [-1.68709909e-01  9.67283185e-01  1.89473496e-01 -1.05602255e-01]
  [ 7.46843357e-01 -2.05128339e-17  6.65000000e-01  1.86710839e-01]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]]


In [ ]:
stanford = Robot("STANFORD")
stanford.calc_invk_and_fk()

Enter the length of links d1 and d2:
d1 = 0.25
d2 = 0.25
Enter the no. of end effector points:1
Enter x, y, z cordinate of point 1:
x = 0.45
y = 0.075
z = 0.1

Inverse kinematics solution: 
First Solution: 
 Theta1 =  0.16514867741462683 
 Theta2 = -0.3176631929822606 
 Extension:  0.23023431780746362 

Second Solution: 
 Theta1 =  3.30674133100442 
 Theta2 = 3.4592558465720535 
 Extension: 0.23023431780746362

Forward kinematics solution:
Final transformation matrix: 
[[[ 9.37042571e-01  1.64398987e-01  3.08097700e-01  0.00000000e+00]
  [ 1.56173762e-01 -9.86393924e-01  5.13496166e-02  0.00000000e+00]
  [ 3.12347524e-01 -1.19401098e-16 -9.49967907e-01  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]]


In [ ]:
puma = Robot("PUMA")
puma.calc_invk_and_fk()

Enter the length of links l1, l2 and l3:
l1 = 0.25
l2 = 0.25
l3 = 0.25
Enter the no. of end effector points:1
Enter x, y, z cordinate of point 1:
x = 0.45
y = 0.075
z = 0.1

Inverse kinematics solution: 
Theta1 =  9.462322208025617 
Theta2 =  -2.0367282034521836 
Theta3 =  -32.328064122162274

forward kinematics solution
Final transformation matrix: 
[[[ 8.37913433e-01  5.30281503e-01 -1.29238563e-01  4.94501864e-01]
  [ 1.09206586e-01  6.91124286e-02  9.91613530e-01  7.34093875e-02]
  [ 5.34766304e-01 -8.45000000e-01  6.12323400e-17  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]]
